# Laboratorio 5
**Threat Hunting**

## Part 1

In [31]:
import json
import pandas as pd
from tld import get_tld

In [32]:
with open('large_eve.json', 'r') as file:
    records = [json.loads(line) for line in file]
print(f"Total records: {len(records)}")

Total records: 746909


In [33]:
dns_records = [record for record in records if record.get('event_type') == 'dns']
print(f"DNS records: {len(dns_records)}")

DNS records: 15749


In [34]:
print("\nTwo random DNS records:")
for record in dns_records[:2]:
    print(json.dumps(record, indent=2))


Two random DNS records:
{
  "timestamp": "2017-07-22T17:33:16.661646-0500",
  "flow_id": 1327836194150542,
  "pcap_cnt": 22269,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 59680,
  "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 15529,
    "rrname": "api.wunderground.com",
    "rrtype": "A",
    "tx_id": 0
  }
}
{
  "timestamp": "2017-07-22T17:33:24.990320-0500",
  "flow_id": 2022925111925872,
  "pcap_cnt": 54352,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 38051,
  "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 58278,
    "rrname": "stork79.dropbox.com",
    "rrtype": "A",
    "tx_id": 0
  }
}


In [35]:
df = pd.json_normalize(dns_records)
print(f"\nDataFrame shape: {df.shape}")


DataFrame shape: (15749, 18)


In [36]:
df_type_a = df[df['dns.rrtype'] == 'A']
print(f"\nType A records: {len(df_type_a)}")


Type A records: 2849


In [37]:
unique_domains = df_type_a['dns.rrname'].unique()
print(f"\nUnique domains: {len(unique_domains)}")


Unique domains: 177


In [38]:
def get_domain_tld(domain):
    """
    Get the domain and TLD from a given domain
    Source: Claude 3.7 Sonnet
    Prompt: Create a function that takes a domain as input and returns the domain and TLD
    """
    try:
        tld = get_tld("https://" + domain, as_object=True)
        return tld.tld
    except:
        return domain.split('.')[-1]

In [39]:
df_tld = pd.DataFrame({'domain': unique_domains})
df_tld['domain_tld'] = df_tld['domain'].apply(get_domain_tld)
print("\nFinal DataFrame with TLDs:")
print(df_tld.head())


Final DataFrame with TLDs:
                                       domain domain_tld
0                        api.wunderground.com        com
1                         stork79.dropbox.com        com
2  hpca-tier2.office.aol.com.ad.aol.aoltw.net        net
3        safebrowsing.clients.google.com.home       home
4                         fxfeeds.mozilla.com        com
